In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import LogCosh

In [2]:
data = pd.read_csv('bitcoin_2017_to_2023.csv',nrows=200000)

In [16]:
features = ['timestamp', 'open', 'high', 'low', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['timestamp'] = data['timestamp'].astype('int64')
X = data[features]
y = data['close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.17),
    Dense(16, activation='relu'),
    Dropout(0.17),
    Dense(1)
])

model.compile(
    optimizer=Nadam(),
    loss=LogCosh(),
    metrics=[RootMeanSquaredError()]
)

history = model.fit(
    X_train, y_train,
    epochs=488,
    batch_size=216,
    validation_data=(X_test, y_test)
)

Epoch 1/488
741/741 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 10160781299548160.0000 - root_mean_squared_error: 17618435810787328.0000 - val_loss: 42177002471424.0000 - val_root_mean_squared_error: 42177086357504.0000
Epoch 2/488
741/741 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 85321781870592.0000 - root_mean_squared_error: 502293136932864.0000 - val_loss: 28712925921280.0000 - val_root_mean_squared_error: 28712976252928.0000
Epoch 3/488
741/741 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 39158894559232.0000 - root_mean_squared_error: 187485472686080.0000 - val_loss: 28856912183296.0000 - val_root_mean_squared_error: 28856981389312.0000
Epoch 4/488
741/741 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 22397281370112.0000 - root_mean_squared_error: 63101634347008.0000 - val_loss: 7558722486272.0000 - val_root_mean_squared_error: 7558739263488.0000
Epoch 5/488
741/741 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 10900441399296.0000 - root_mean_squared_error: 13921670922240.0000 - val_loss: 1482907503